# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [155]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [156]:
tot_schools = len((school_data_complete["school_name"]).unique())
tot_students = len(school_data_complete["student_name"].unique())
tot_budget = school_data_complete["budget"].sum()
avg_math = school_data_complete["math_score"].mean()
avg_reading = school_data_complete["reading_score"].mean()
overall_avg_score = (avg_math + avg_reading)/2

#Percentage of students with passing math score 
math_score = school_data_complete["math_score"] > 69.99
math_pass_rate = (math_score.value_counts("True")) * 100
math_pass_rate = float(math_pass_rate[True])

#Percentage of students with passing reading score
reading_score = school_data_complete["reading_score"] > 69.99
reading_pass_rate = (reading_score.value_counts(True)) * 100
reading_pass_rate = float(reading_pass_rate[True])


district_df = pd.DataFrame({"Total Schools":[tot_schools],
                           "Total Students":[tot_students],
                           "Total Budget":[tot_budget],
                           "Average Math Score":[avg_math],
                           "Average Reading Score":[avg_reading],
                           "Overall Average Score": [overall_avg_score],
                           "Math Pass Rate": [math_pass_rate],
                           "Reading Pass Rate":[reading_pass_rate]})
district_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,Math Pass Rate,Reading Pass Rate
0,15,32715,82932329558,78.985371,81.87784,80.431606,74.980853,85.805463


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [169]:
avg_scores = school_data_complete.drop(['Student ID', 'School ID', 'size', 'budget'], axis=1).groupby('school_name').mean()

num_student = school_data_complete.drop(['Student ID', 'gender', 'grade', 'School ID', 'math_score', 'reading_score', 'type', 'size', 'budget'], axis=1).groupby('school_name').count()

df1 = pd.merge(avg_scores, num_student, on="school_name", how="outer")

budget_tot = school_data_complete.drop(['Student ID', 'gender', 'grade', 'reading_score', 'math_score', 'School ID', 'type', 'size', 'student_name'], axis=1).groupby('school_name').max()

df2 = pd.merge(df1, budget_tot, on='school_name', how='outer')

typeofschool = school_data_complete.drop(['Student ID', 'gender', 'grade', 'reading_score', 'math_score', 'School ID', 'size', 'budget', 'student_name'], axis=1).groupby('school_name').min()

budget_per_student = df2['budget'] / df2['student_name']

df2['Budget/Student'] = budget_per_student

df2['Type'] = typeofschool

df2 = df2.rename(columns={'student_name':'Total Students', "reading_score":"Average Reading Score", 'math_score':'Average Math Score', 'budget':'Total Budget', 'Budget/Student': 'Budget per Student'})
df2


#Group
math_group = school_data_complete[school_data_complete["math_score"]>70].groupby(["school_name"], as_index=False)
pass_math = pd.DataFrame(math_group["math_score"].count())
pass_math.columns= ["school_name", "pass_math"]

read_group = school_data_complete[school_data_complete["reading_score"]>70].groupby(["school_name"], as_index=False)
pass_read = (pd.DataFrame(read_group["reading_score"].count()))
pass_read.columns= ["school_name", "pass_read",]

df3 = pd.merge(df2, pass_read, on="school_name", how="left")
df3

df4 = pd.merge(df3, pass_math, on='school_name', how='left')


math_pass_percent = (df4['pass_math'] / df4['Total Students'])*100
read_pass_percent = (df4['pass_read'] / df4['Total Students'])*100
overall_pass_rate = (math_pass_percent + read_pass_percent)/2
df4["% Passing Math"] = math_pass_percent
df4["% Passing Reading"] = read_pass_percent
df4["% Overall Passing"] = overall_pass_rate
df4.drop(['pass_read', 'pass_math'], axis=1)
summary = df4
summary




,school_name,Average Reading Score,Average Math Score,Total Students,Total Budget,Budget per Student,Type,pass_read,pass_math,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,81.033963,77.048432,4976,3124928,628.0,District,3946,3216,64.630225,79.300643,71.965434
1,Cabrera High School,83.975780,83.061895,1858,1081356,582.0,Charter,1744,1664,89.558665,93.864370,91.711518
2,Figueroa High School,81.158020,76.711767,2949,1884411,639.0,District,2313,1880,63.750424,78.433367,71.091896
3,Ford High School,80.746258,77.102592,2739,1763916,644.0,District,2123,1801,65.753925,77.510040,71.631982
4,Griffin High School,83.816757,83.351499,1468,917500,625.0,Charter,1371,1317,89.713896,93.392371,91.553134
5,Hernandez High School,80.934412,77.289752,4635,3022020,652.0,District,3624,3001,64.746494,78.187702,71.467098
6,Holden High School,83.814988,83.803279,427,248087,581.0,Charter,396,387,90.632319,92.740047,91.686183
7,Huang High School,81.182722,76.629414,2917,1910635,655.0,District,2299,1847,63.318478,78.813850,71.066164
8,Johnson High School,80.966394,77.072464,4761,3094650,650.0,District,3727,3040,63.852132,78.281874,71.067003
9,Pena High School,84.044699,83.839917,962,585858,609.0,Charter,887,882,91.683992,92.203742,91.943867


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [175]:
# Create sorted list of top performing schools
sort_school_summary = summary.sort_values("% Overall Passing", ascending=False)
sort_school_summary.head(5)

,school_name,Average Reading Score,Average Math Score,Total Students,Total Budget,Budget per Student,Type,pass_read,pass_math,% Passing Math,% Passing Reading,% Overall Passing
13,Wilson High School,83.989488,83.274201,2283,1319574,578.0,Charter,2129,2076,90.932983,93.254490,92.093736
9,Pena High School,84.044699,83.839917,962,585858,609.0,Charter,887,882,91.683992,92.203742,91.943867
14,Wright High School,83.955000,83.682222,1800,1049400,583.0,Charter,1682,1625,90.277778,93.444444,91.861111
1,Cabrera High School,83.975780,83.061895,1858,1081356,582.0,Charter,1744,1664,89.558665,93.864370,91.711518
6,Holden High School,83.814988,83.803279,427,248087,581.0,Charter,396,387,90.632319,92.740047,91.686183


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [176]:
sort_school_summary = summary.sort_values("% Overall Passing", ascending=True)[:5]
sort_school_summary.head(5)

,school_name,Average Reading Score,Average Math Score,Total Students,Total Budget,Budget per Student,Type,pass_read,pass_math,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,80.744686,76.842711,3999,2547363,637.0,District,3109,2562,64.066017,77.744436,70.905226
7,Huang High School,81.182722,76.629414,2917,1910635,655.0,District,2299,1847,63.318478,78.813850,71.066164
8,Johnson High School,80.966394,77.072464,4761,3094650,650.0,District,3727,3040,63.852132,78.281874,71.067003
2,Figueroa High School,81.158020,76.711767,2949,1884411,639.0,District,2313,1880,63.750424,78.433367,71.091896
5,Hernandez High School,80.934412,77.289752,4635,3022020,652.0,District,3624,3001,64.746494,78.187702,71.467098


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [178]:
nineth = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth = school_data_complete[(school_data_complete["grade"] == "12th")]

nineth_scores = nineth.groupby(["school_name"]).mean()["math_score"]
tenth_scores = tenth.groupby(["school_name"]).mean()["math_score"]
eleventh_scores = eleventh.groupby(["school_name"]).mean()["math_score"]
twelfth_scores = twelfth.groupby(["school_name"]).mean()["math_score"]


grade_scores = pd.DataFrame({"9th": nineth_scores,
                                "10th":tenth_scores,
                                "11th":eleventh_scores,
                                "12th":twelfth_scores})



grade_scores.index.name = None

grade_scores = grade_scores [["9th", "10th", "11th", "12th"]]


grade_scores

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [179]:
nineth = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth = school_data_complete[(school_data_complete["grade"] == "12th")]

nineth_scores = nineth.groupby(["school_name"]).mean()["reading_score"]
tenth_scores = tenth.groupby(["school_name"]).mean()["reading_score"]
eleventh_scores = eleventh.groupby(["school_name"]).mean()["reading_score"]
twelfth_scores = twelfth.groupby(["school_name"]).mean()["reading_score"]


grade_scores = pd.DataFrame({"9th": nineth_scores,
                                "10th":tenth_scores,
                                "11th":eleventh_scores,
                                "12th":twelfth_scores})



grade_scores.index.name = None

grade_scores = grade_scores [["9th", "10th", "11th", "12th"]]


grade_scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [182]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
scoreByBudget = summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].groupby(pd.cut(summary["Budget per Student"], bins=spending_bins, labels=group_names )).mean()
scoreByBudget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Budget per Student,,,,,
<$585,83.455399,83.933814,90.350436,93.325838,91.838137
$585-615,83.599686,83.885211,90.788049,92.410786,91.599418
$615-645,79.079225,81.891436,73.021426,83.214343,78.117884
$645-675,76.997210,81.027843,63.972368,78.427809,71.200088


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [183]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
scoreBySize = summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].groupby(pd.cut(summary["Total Students"], bins=size_bins, labels=group_names )).mean()
scoreBySize.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Students,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [185]:
bins = [0, 1, 2]
group_names = ["Charter", "District"]
scoreByType = summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].groupby(pd.cut(summary["Type"], bins=bins, labels=group_names )).mean()
scoreByType.head()

TypeError: '<' not supported between instances of 'int' and 'str'